Esse notebook tem por objetivo fazer a relação de empresas que importam somente uma marca, facilitando a relação importador-

In [3]:
# Importing the modules needed
import sys

import pandas as pd

sys.path.append("../src/")

from src.data.dremio_utils import *
# Data Handling
from dotenv import dotenv_values 

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType

In [4]:
spark = SparkSession.builder.master("local[1]").appName("attributes_dict").getOrCreate()

In [5]:
config = dotenv_values(".env")
bds = BaseDremioService(config)

## 1. Getting Merged Data

In [6]:
grouped_data = spark.read.parquet("../data/processed/average_unity_price_historic.parquet")

In [7]:
grouped_data = grouped_data.groupBy(['ncm', 'importador_uf', 'importador_municipio', 'urf', 'id_pais_origem', 'ano', 'semestre']).avg('avg_valor_item')

# 2. Gabarito de datas e combinações 

In [8]:
unique_combinations = grouped_data.select('ncm', 'importador_uf', 'importador_municipio', 'urf', 'id_pais_origem').distinct()
unique_combinations.show(5)

+---------+--------------------+--------------------+--------------------+--------------+
|      ncm|       importador_uf|importador_municipio|                 urf|id_pais_origem|
+---------+--------------------+--------------------+--------------------+--------------+
|     null|              TOLEDO|                  PR|     PORTO DE SANTOS|    BANGLADESH|
|2023000.0|             GOIANIA|                  GO|IRF - PORTO DE SUAPE|     AUSTRÁLIA|
|3055310.0|CABO DE SANTO AGO...|                  PE|              ITAJAI|       NORUEGA|
|4071900.0|               BAURU|                  SP|AEROPORTO INTERNA...|ESTADOS UNIDOS|
|6029029.0|                  SP|            HOLAMBRA|AEROPORTO INTERNA...|       HOLANDA|
+---------+--------------------+--------------------+--------------------+--------------+


In [9]:
anos_data = [{"ano": 2018}, {"ano": 2019}, {"ano": 2020}, {"ano": 2021}, {"ano": 2022}, {"ano": 2023}]
anos_schema =StructType([StructField("ano", IntegerType())])
 
years_df = spark.createDataFrame(data=anos_data, schema=anos_schema)

semestres_data = [{"semestre": 1}, {"semestre": 2}]
semestres_schema =StructType([StructField("semestre", IntegerType())])

semesters_df = spark.createDataFrame(data=semestres_data, schema=semestres_schema)

gabarito_datas = years_df.crossJoin(semesters_df)
gabarito_datas.show(5)

+----+--------+
| ano|semestre|
+----+--------+
|2018|       1|
|2018|       2|
|2019|       1|
|2019|       2|
|2020|       1|
+----+--------+


In [10]:
gabarito_comb = unique_combinations.crossJoin(gabarito_datas)
gabarito_comb.show(5)

+----+-------------+--------------------+---------------+--------------+----+--------+
| ncm|importador_uf|importador_municipio|            urf|id_pais_origem| ano|semestre|
+----+-------------+--------------------+---------------+--------------+----+--------+
|null|       TOLEDO|                  PR|PORTO DE SANTOS|    BANGLADESH|2018|       1|
|null|       TOLEDO|                  PR|PORTO DE SANTOS|    BANGLADESH|2018|       2|
|null|       TOLEDO|                  PR|PORTO DE SANTOS|    BANGLADESH|2019|       1|
|null|       TOLEDO|                  PR|PORTO DE SANTOS|    BANGLADESH|2019|       2|
|null|       TOLEDO|                  PR|PORTO DE SANTOS|    BANGLADESH|2020|       1|
+----+-------------+--------------------+---------------+--------------+----+--------+


In [11]:
df_2b_filled = gabarito_comb.join(grouped_data, ['ncm', 'importador_uf', 'importador_municipio', 'urf', 'id_pais_origem', 'ano', 'semestre'], "left")
df_2b_filled.head(15)

Py4JJavaError: An error occurred while calling o88.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 11.0 failed 1 times, most recent failure: Lost task 3.0 in stage 11.0 (TID 834, 192.168.0.7, executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2008)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2007)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2007)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handle

In [ ]:
df_2b_filled = df_2b_filled.withColumnRenamed("avg(avg_valor_item)", "avg_valor_item")

In [ ]:
df_2b_filled.write.parquet("../data/processed/data_2b_filled")

In [ ]:
del df_2b_filled, gabarito_comb, grouped_data

In [ ]:
import os
from tqdm import tqdm

# Caminho para a pasta contendo os arquivos Parquet
caminho = '../data/processed/data_2b_filled/'

# Lista de todos os arquivos Parquet
arquivos_parquet = [arq for arq in os.listdir(caminho) if arq.lower().endswith('.parquet')]

concat_df = pd.DataFrame()
with tqdm(desc="concatenating parquet", total=len(arquivos_parquet)) as pbar:
    for arquivo in arquivos_parquet:
        df_aux = pd.read_parquet(caminho+arquivo)
        concat_df = pd.concat([concat_df,df_aux])
        pbar.update(1)

## 3. Inference of the value for the first semesters of 2024

### 3.1 Inference using linear interpolation

In [ ]:
years_df = pd.DataFrame.from_dict({"ano": [2018, 2019, 2020, 2021, 2022, 2023, 2024]})
semesters_df = pd.DataFrame.from_dict({"semestre": [1, 2]})
gabarito_datas = years_df.join(semesters_df, how="cross")
gabarito_comb = unique_combinations.join(gabarito_datas, how="cross")

In [ ]:
df_2b_infer = gabarito_comb.merge(df_filled, how="left", on=['ncm', 'importador_uf', 'importador_municipio', 'urf', 'name_pt', 'ano', 'semestre'])
df_2b_infer.head()

In [ ]:
df_2b_infer["avg_valor_item"] = df_2b_infer["avg_valor_item"].interpolate()

In [ ]:
df_2b_infer["anosem"] = df_2b_infer["ano"].astype(str) + df_2b_infer["semestre"].astype(str) 

In [ ]:
for _, df in df_2b_infer.groupby(['ncm', 'importador_uf', 'importador_municipio', 'urf', 'name_pt']):
    print(df["avg_valor_item"].values)
    df.plot(y="avg_valor_item", x="anosem")
    break

### 3.1 Inference using linear interpolation